In [186]:
import os
import json
import gzip
import pandas as pd
from urllib.request import urlopen
from collections import defaultdict
from sklearn import linear_model

# import working with data libraries
import pandas as pd
import numpy as np

# import seaborn
import seaborn as sns

# import matplotlib
import matplotlib.pyplot as plt # Typical way of import MPL
import matplotlib as mpl # This line is used less frequently

In [3]:
### load the review data

review_data = []
with gzip.open('Software.json.gz') as f:
    for l in f:
        review_data.append(json.loads(l.strip()))
    
# total length of list, this number equals total number of reviews
print(len(review_data))

# first row of the list
print(review_data[10])

459436
{'overall': 2.0, 'vote': '3', 'verified': True, 'reviewTime': '05 3, 2009', 'reviewerID': 'A1CUJ6ZT82H7LS', 'asin': '0132147556', 'style': {'Format:': ' Paperback'}, 'reviewerName': 'Brian', 'reviewText': "Maybe it's just me (I have no marketing background but desperately want to learn for my start-up) but I cannot get hardly anything out of this text. I have tried very hard to tread through the writing and learn something useful but chapter after chapter seems to be the exact same thing... overly wordy, rambling & unnecessarily academic writing with no comprehensible message behind it, no strategies I can apply to my own business and no organization that I can make sense of. Feels almost like I'm reading a paper a college student BSed their way through with a whole bunch of long words and cryptic sentences in order to sound impressive. What a rip-off. I paid $70 for this and it has been the most useless book I've paid for in my research process (and I've read 16 other books thu

In [131]:
# convert list into pandas dataframe

all_products = {set}
df_review = pd.DataFrame.from_dict(review_data)

#filter out the non-verified reviews, and leave only the columns we want
df_review = df_review[df_review['verified']==True][["overall", "reviewTime", "asin", "reviewText", "summary"]]
df_review = df_review.rename(columns={"asin": "product_id", "overall": "rating"})
print(len(df_review))
df_review.head()

309345


,rating,reviewTime,product_id,reviewText,summary
0,4.0,"03 11, 2014",0077613252,The materials arrived early and were in excell...,Material Great
1,4.0,"02 23, 2014",0077613252,I am really enjoying this book with the worksh...,Health
2,1.0,"02 17, 2014",0077613252,"IF YOU ARE TAKING THIS CLASS DON""T WASTE YOUR ...",ARE YOU KIDING ME?
3,3.0,"02 17, 2014",0077613252,This book was missing pages!!! Important pages...,missing pages!!
5,4.0,"08 15, 2011",0132147556,"Strong backgroung, good read, quite up to date...",E-marketing 6th ed


In [63]:
### load the mega data

mega_data = []
with gzip.open('meta_Software.json.gz') as f:
    for l in f:
        mega_data.append(json.loads(l.strip()))
    
# total length of list, this number equals total number of products
print(len(mega_data))

# first row of the list
print(mega_data[1])

26790
{'category': [], 'tech1': '', 'description': ['', '<b>Latin rhythms that will get your kids singing in Spanish</b>', '<i>Sing, Watch, and Learn Spanish</i> helps your kids ages four through eight take a giant step in learning Spanish by combining two time-honored methods kids have always used to develop their language skills: Imitating other kids and singing along with simple melodies. This charming DVD contains 16 music videos featuring kids engaged in fun activities, from visiting animals at the zoo to comparing clothing sizes in grandmas closet.', 'Each video features an original song of authentic Latin rhythms that gets kids singing along with the children on screen. As they watch, listen, and sing along, kids absorb 300 Spanish words, each of which is shouted out in a song and displayed as a subtitle on screen.', '', '', ''], 'fit': '', 'title': 'Sing, Watch, &amp; Learn Spanish (DVD + Guide): 16 Fun Music Videos to Introduce Your Child to Spanish', 'also_buy': [], 'tech2': 

In [64]:
# convert list into pandas dataframe

df_product = pd.DataFrame.from_dict(mega_data)

#filter out the non-verified reviews, and leave only the columns we want
df_product = df_product[["asin", "rank"]]
df_product = df_product.rename(columns={"asin": "product_id"})
print(len(df_product))
df_product.head()
# TODO:filter out the products that has reviews, rank in software
# if(product_id)
# df_software = df_product[df_product['rank']]

26790


,product_id,rank
0,0030672120,"25,550 in Software ("
1,0071480935,"15,792 in Software ("
2,007329506X,"16,900 in Software ("
3,0073513458,"12,986 in Software ("
4,0073525758,"14,861 in Software ("


In [65]:
df_product = df_product.dropna(axis='index')
print(len(df_product))
df_product = df_product[df_product['rank'].str.contains('Software', na=False)]
print(len(df_product))


26790
21316


In [68]:
import string
def toInt(str_in):
    try:
        str_in=''.join(i for i in str_in if i.isdigit())
        output = str_in
    except:
        output = np.nan
        
    return output

In [69]:
df_product['rank'] = df_product['rank'].apply(toInt)
df_product.head()

,product_id,rank
0,0030672120,25550
1,0071480935,15792
2,007329506X,16900
3,0073513458,12986
4,0073525758,14861


In [84]:
all_products_Y = {set}

for index, product_id, rank in df_product.itertuples():
    all_products_Y.add(product_id)
# print(all_products_Y)
print(len(all_products_Y))

17142


In [132]:
options = all_products_Y
  
# selecting rows that product ids are included in the product dataframe
df_review = df_review[df_review['product_id'].isin(options)]

In [127]:
np.random.seed(32) # for reproducible results.
df_product_x1 = np.array_split(df_product.reindex(np.random.permutation(df_product.index)),10)[0]
df_product_x2 = np.array_split(df_product.reindex(np.random.permutation(df_product.index)),10)[1]
df_product_x3 = np.array_split(df_product.reindex(np.random.permutation(df_product.index)),10)[2]
df_product_x4 = np.array_split(df_product.reindex(np.random.permutation(df_product.index)),10)[3]
df_product_x5 = np.array_split(df_product.reindex(np.random.permutation(df_product.index)),10)[4]
df_product_x6 = np.array_split(df_product.reindex(np.random.permutation(df_product.index)),10)[5]
df_product_x7 = np.array_split(df_product.reindex(np.random.permutation(df_product.index)),10)[6]
df_product_x8 = np.array_split(df_product.reindex(np.random.permutation(df_product.index)),10)[7]
df_product_x9 = np.array_split(df_product.reindex(np.random.permutation(df_product.index)),10)[8]
#test set
df_product_y = np.array_split(df_product.reindex(np.random.permutation(df_product.index)),10)[9]
#training set
df_product_x = pd.concat([df_product_x1, df_product_x2, df_product_x3, df_product_x4, df_product_x5, df_product_x6, df_product_x7, df_product_x8, df_product_x9])
print(len(df_product), len(df_product_x), len(df_product_y), (len(df_product_x) + len(df_product_y)))
df_product.head()

21316 19185 2131 21316


,product_id,rank
0,0030672120,25550
1,0071480935,15792
2,007329506X,16900
3,0073513458,12986
4,0073525758,14861


In [135]:
all_products = {set}
product_rank = {}
y = []
for index, product_id, rank in df_product_x.itertuples():
    all_products.add(product_id)   
    product_rank.update({product_id: rank})
 

print(product_rank['0030672120'])

25550


In [172]:
ratingsPerItem = defaultdict(list)
for index, rating, reviewTime, product_id, reviewText, summary in df_review.itertuples():
    if(product_id in all_products):
        ratingsPerItem[product_id].append(rating)
print(len(all_products))
print(len(df_review))
df_review.head()

print(ratingsPerItem['0077613252'])
print(len(ratingsPerItem))
print(s)

11532
187900
[4.0, 4.0, 1.0, 3.0, 5.0, 4.0, 4.0, 4.0, 5.0, 2.0, 1.0]
9049
121555


In [196]:
Xtrain = []
ytrain = []


def feature(datum):
    Xtrain.append(datum[1])
    ytrain.append(int(product_rank[datum[0]]))
    feat = []
    feat.append(1)
    feat.append(len(datum[1])) # offset
    feat.append(sum(datum[1])/len(datum[1]))
    return feat

X = [feature(d) for d in ratingsPerItem.items()]

9049

In [ ]:
mod = linear_model.LogisticRegression(C=1, max_iter=3000)
mod.fit(X, ytrain) 

# pred = mod.predict(Xvalid)
# correct = pred == yvalid
# sum(correct) / len(correct)

In [185]:
# print(Xtrain)
# print(ytrain)
print(len(Xtrain), len(ytrain))

9049 9049
